# Example Use of AdvanceOrder

## Setup

In [1]:
import os
os.chdir(os.path.pardir)

# Testing classes
from src.market.spot import Spot
from src.market_actor import MarketActorStub
from src.market_listener import MarketListenerStub

In [2]:
transact_fee_rate = 0.2 / 100

market = Spot("ETH", "USD")
market_listener = MarketListenerStub(market)
market_actor = MarketActorStub(transact_fee_rate)

In [3]:
stop_loss_rate = 2 / 100

## Trailing Stop Loss

In [4]:
from src.advance_order.trailing_stop_loss import TrailingStopLoss

entry_price = 1000
trailing_rate = 1 / 100

### Long Position

In [5]:
long_position = market_actor.open_position(market, price=entry_price, size=10)

trailing_stop_loss = TrailingStopLoss(long_position, market_actor, market_listener,
        entry_price, trailing_rate, use_orderbook=False)

f"trigger_price = {trailing_stop_loss.stop_loss_price}"

Bought 10.00 units of ETH/USD    at 1000.00


'trigger_price = 990.0'

In [6]:
# Not triggered: change in stop_loss_price
market_listener.set_current_price(1100)
trailing_stop_loss.update()

f"trigger_price = {trailing_stop_loss.stop_loss_price}"

'trigger_price = 1089.0'

In [7]:
# Not triggered: no change in stop_loss_price
market_listener.set_current_price(1090)
trailing_stop_loss.update()

f"trigger_price = {trailing_stop_loss.stop_loss_price}"

'trigger_price = 1089.0'

In [8]:
# Triggered
market_listener.set_current_price(1080)
trailing_stop_loss.update()

print(f"Order filled = {trailing_stop_loss.filled}")
# View ending position
trailing_stop_loss.position.balances

Sold   9.98 units of ETH/USD    at 1080.00
Order filled = True


{'ETH': 0.0, 'USD': 756.8431999999993}

### Short Position

In [9]:
short_position = market_actor.open_position(market, price=entry_price, size=-10)

trailing_stop_loss = TrailingStopLoss(short_position, market_actor, market_listener,
        entry_price, trailing_rate, use_orderbook=False)

f"trigger_price = {trailing_stop_loss.stop_loss_price}"

Sold   10.00 units of ETH/USD    at 1000.00


'trigger_price = 1010.0'

In [10]:
# Not triggered: change in stop_loss_price
market_listener.set_current_price(900)
trailing_stop_loss.update()

f"trigger_price = {trailing_stop_loss.stop_loss_price}"

'trigger_price = 909.0'

In [11]:
# Triggered
market_listener.set_current_price(950)
trailing_stop_loss.update()

print(f"Order filled = {trailing_stop_loss.filled}")
# View ending position
trailing_stop_loss.position.balances

Bought 10.02 units of ETH/USD    at 950.00
Order filled = True


{'ETH': 0.0, 'USD': 460.9619238476953}

## Convertible Stop Loss

In [12]:
from src.advance_order.convertible_stop_loss import ConvertibleStopLoss

In [13]:
long_position = market_actor.open_position(market, price=entry_price, size=10)

convertible_stop_loss = ConvertibleStopLoss(long_position, market_actor, market_listener,
        entry_price, stop_loss_rate, trailing_rate, use_orderbook=False, stop_loss_as_rate=True)

print(f"trigger_price = {convertible_stop_loss.stop_loss_price}")
(1 - convertible_stop_loss.stop_loss_price / entry_price) * 100

Bought 10.00 units of ETH/USD    at 1000.00
trigger_price = 980.0


2.0000000000000018

In [14]:
# Not triggered: change in stop_loss_price to trailing rate
market_listener.set_current_price(1050)
convertible_stop_loss.update()

print(f"trigger_price = {convertible_stop_loss.stop_loss_price}")
(1 - convertible_stop_loss.stop_loss_price / 1050) * 100

trigger_price = 1039.5


1.0000000000000009

In [15]:
# Triggered
market_listener.set_current_price(1020)
convertible_stop_loss.update()

print(f"Order filled = {convertible_stop_loss.filled}")
# View ending position
convertible_stop_loss.position.balances

Sold   9.98 units of ETH/USD    at 1020.00
Order filled = True


{'ETH': 0.0, 'USD': 159.24079999999958}